<a href="https://colab.research.google.com/github/datadigger01/AI-Trade/blob/main/Bertopic_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 1. Package Installation
# ==========================================

!pip install bertopic
#!pip install --upgrade gensim

# Choose an embedding backend
!pip install bertopic[flair,gensim,spacy,use,visualization]

# Topic modeling with images
# !pip install bertopic[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 

In [ ]:
!pip install konlpy
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.5 MB/s eta 0:00:00


In [ ]:
"""
한국어 텍스트 데이터를 위한 BERTopic 모델링
파일: trade_success_case_data.csv의 bdtCntnt 컬럼 분석
"""

import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from konlpy.tag import Okt, Hannanum, Kkma, Komoran
#from konlpy.tag import Okt

from sklearn.cluster import KMeans, AgglomerativeClustering
from umap import UMAP
from hdbscan import HDBSCAN

import string
import re
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


#### ============================================
#### 1. 데이터 로드
#### ============================================

In [ ]:
# ============================================
# 1. 데이터 로드
# ============================================
print("=" * 60)
print("1. 데이터 로드 중...")
print("=" * 60)

df = pd.read_csv('https://raw.githubusercontent.com/datadigger01/AI-Trade/main/Data/trade_success_case_data.csv')
df['othbcDt'] = pd.to_datetime(df['othbcDt'])

print(f"전체 데이터 수: {len(df)}")

# bdtCntnt 컬럼 추출 및 결측치 제거
texts = df['bdtCntnt'].dropna().tolist()
timestamps = df['othbcDt'].dt.year.to_list() # 토픽별 Time Series를 위한 년단위 timestamps

print(f"유효한 텍스트 수: {len(texts)}")
print(f"평균 텍스트 길이: {np.mean([len(text) for text in texts]):.0f}자\n")
print(f"데이터셋 정보:{df.info()}")

1. 데이터 로드 중...
전체 데이터 수: 194
유효한 텍스트 수: 194
평균 텍스트 길이: 2746자

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   bdtCntnt   194 non-null    object        
 1   dataType   194 non-null    object        
 2   ovrofInfo  0 non-null      float64       
 3   dmgeAmt    0 non-null      float64       
 4   fraudType  0 non-null      float64       
 5   othbcDt    194 non-null    datetime64[ns]
 6   bbstxSn    194 non-null    int64         
 7   titl       194 non-null    object        
 8   natn       194 non-null    object        
 9   regn       194 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 15.3+ KB
데이터셋 정보:None


In [ ]:
texts[11]
#timestamps[3]

'국내 최초 UV-C LED 칫솔 살균기 개발2000년 설립된 비에네스소프트는 소프트웨어 개발 기업이자 \'울트라웨이브\'라는 브랜드로살균에 초점을 맞춘 퍼스널 헬스케어 제품을 개발&middot;제조&middot;수출하는 기업이다.독립채산제로 운영되는 특성상 회사 내 아이담 사업부에서 \'울트라웨이브\' 사업을 총괄한다.원래 아이담 사업부에서는 보이스레코더와 MP3 같은 음향기기를 대부분 ODM 방식으로납품해 왔다. 스마트폰 사용이 보편화되면서 음향기기 시장 규모가 줄어들자 친환경 반도체 광원인UV LED로 눈을 돌려 2018년 국내 최초로 UV-C LED 칫솔 살균기 개발에 성공했다.기존 UV램프 칫솔 살균기는 UV램프 안에 수은이 들어 있고, 살균 과정에서 오존을 발생기키기때문에 비릿한 냄새가 나는 단점이 있었다. UV LED 파장 중 UV-C를 사용한 울트라웨이브칫솔 살균기는 3분이면 뮤탄스균&middot;녹농균&middot;대장균&middot;황색포도상구균을 99.9%를 살균한다.수은을 함유하고 있지 않아 친환경적이고, 오존을 발생시키지 않아 불쾌감 없이 쓸 수 있다.EU의 전기&middot;전자제품 유해물질사용제한(RoHS) 인증을 취득했고,미국 FDA 승인, 미국 FOC 인증, 유럽 CD 인증도 완료했다.화상상담으로 해외 시장 확대 가능성을 보다비에네스소프트는 칫솔 살균기, 면도기 살균기, 마스크 살균기 등 UV LED 살균기 라인과 함께EARPET(반려동물의 귀 질환&middot;염증을 치료하는 동물의료기기,PEPPI3(반려동물 전용 MP3) 등펫헬스케어 라인을 구성했지만 2020년, 코로나19라는 변수를 맞았다.코로나19로 참가 예정이었던 해외 전시회가 줄줄이 취소되는 상황이 발생한 것이다.   "2019년에 해외 시장 조사와 신규 바이어 영업을 위해 KOTRA에 도움을 요청했다면, 2020년에는 영국, 스페인, 독일, 미국, 싱가포르, 홍콩, 말레이시아, 태국, 칠레 등 여러 국가의 기업들과 화상상담회를 진행했습니다. 해외 시장을 확대하려는 중

#### ============================================
#### 2. 텍스트 전처리
#### ============================================

In [ ]:
# ============================================
# 2. 텍스트 전처리
# ============================================
print("=" * 60)
print("2. 텍스트 전처리 중...")
print("=" * 60)

def combine_specific_words(text, word_pairs):
    """특정 단어 조합을 하나로 합치는 함수"""
    for word_pair in word_pairs:
        # 공백이 있는 형태를 공백 없는 형태로 변환
        text = text.replace(word_pair, word_pair.replace(' ', ''))
    return text

def replace_specific_words(text, word_dict):
    """특정 단어를 다른 단어로 치환하는 함수"""
    for old_word, new_word in word_dict.items():
        text = text.replace(old_word, new_word)
    return text

def preprocess_korean_text(text):
    """한국어 텍스트 전처리"""
    if pd.isna(text):
        return ""  ## doucument가 결측인경우 NaN return

    #########  특수문자 포함 정제처리할 단어 처리(삭제)
    # HTML 태그 제거
    pattern = r'<[^>]*>'
    text = re.sub(pattern, '', text)
    ## HTML 태그 제거
    ##text = re.sub(r'&[a-z]+;', ' ', text)

    # E-mail제거
    pattern = r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern, '', text)

    # URL제거
    pattern = r'(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern, '', text)
    ## Remove URLs
    # text = re.sub(r'http\S+|www.\S+', '', text)

    # 특수문자 제거 (한글, 영문, 공백만 유지)
    text = re.sub(r'[^가-힣a-zA-Z\s]', ' ', text)
    #text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)

    # 한글 자음, 모음 제거
    pattern = r'([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern, '', text)

    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # # 연속 공백 정리
    # text = ' '.join(text.split())

    # 특수기호제거
    #pattern = r'[^\w\s]'
    text = re.sub(r'[^\w\s]', '', text)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 영어는 모두 대문자로 전환
    text = text.upper()

    ###########################################################################
    ##  특정 단어에 대한 변환, 조합 및 제거
    ###########################################################################
    # 토큰화 전에 특정 단어를 다른 단어로 치환
    word_replacements = {
        'KOTRA': '코트라',
        '어플리케이션': '애플리케이션',
        '카메라': '캠코더',  # 예시
        # 필요한 단어 치환을 여기에 추가하세요
    }
    text = replace_specific_words(text, word_replacements)

    # 토큰화 전에 특정 단어 조합을 먼저 결합
    word_pairs_to_combine = [
        '성공 사례',
        '중소 기업',
        '수출 전략',
        '시장 조사',
        '데이터 분석',
        '제품 개발',
        '생활 용품'
        # 필요한 단어 조합을 여기에 추가하세요
    ]
    text = combine_specific_words(text, word_pairs_to_combine)

    # 특정 단어 제거
    remove_words = ['바로가기',
                    '성공사례',
                    'RSQUO','LSQUO','LDQUO','RDQUO',
                    '원문 슬기로운 코트라 활용법',
                    '수출전문위원이 전하는 중소기업 수출성공스토리'
                    ]
    for word in remove_words:
        text = text.replace(word, '')

    return text.strip()

# 전처리 적용
cleaned_texts = [preprocess_korean_text(text) for text in texts]
print("전처리 완료!\n")

2. 텍스트 전처리 중...
전처리 완료!



In [ ]:
cleaned_texts[11]

'국내 최초 UV C LED 칫솔 살균기 개발 년 설립된 비에네스소프트는 소프트웨어 개발 기업이자 울트라웨이브 라는 브랜드로살균에 초점을 맞춘 퍼스널 헬스케어 제품을 개발 MIDDOT 제조 MIDDOT 수출하는 기업이다 독립채산제로 운영되는 특성상 회사 내 아이담 사업부에서 울트라웨이브 사업을 총괄한다 원래 아이담 사업부에서는 보이스레코더와 MP 같은 음향기기를 대부분 ODM 방식으로납품해 왔다 스마트폰 사용이 보편화되면서 음향기기 시장 규모가 줄어들자 친환경 반도체 광원인UV LED로 눈을 돌려 년 국내 최초로 UV C LED 칫솔 살균기 개발에 성공했다 기존 UV램프 칫솔 살균기는 UV램프 안에 수은이 들어 있고 살균 과정에서 오존을 발생기키기때문에 비릿한 냄새가 나는 단점이 있었다 UV LED 파장 중 UV C를 사용한 울트라웨이브칫솔 살균기는 분이면 뮤탄스균 MIDDOT 녹농균 MIDDOT 대장균 MIDDOT 황색포도상구균을 를 살균한다 수은을 함유하고 있지 않아 친환경적이고 오존을 발생시키지 않아 불쾌감 없이 쓸 수 있다 EU의 전기 MIDDOT 전자제품 유해물질사용제한 ROHS 인증을 취득했고 미국 FDA 승인 미국 FOC 인증 유럽 CD 인증도 완료했다 화상상담으로 해외 시장 확대 가능성을 보다비에네스소프트는 칫솔 살균기 면도기 살균기 마스크 살균기 등 UV LED 살균기 라인과 함께EARPET 반려동물의 귀 질환 MIDDOT 염증을 치료하는 동물의료기기 PEPPI 반려동물 전용 MP 등펫헬스케어 라인을 구성했지만 년 코로나 라는 변수를 맞았다 코로나 로 참가 예정이었던 해외 전시회가 줄줄이 취소되는 상황이 발생한 것이다 년에 해외 시장조사와 신규 바이어 영업을 위해 코트라에 도움을 요청했다면 년에는 영국 스페인 독일 미국 싱가포르 홍콩 말레이시아 태국 칠레 등 여러 국가의 기업들과 화상상담회를 진행했습니다 해외 시장을 확대하려는 중소기업 특히 신제품을 개발한 기업에게는 해외 전시회 참여가 필수인데 코로나 로 인해 해외 출장마저 갈 수 없는 막막한

#### ============================================
#### 3. 한국어 형태소 분석 및 불용어 처리
#### ============================================

In [ ]:
# ============================================
# 3. 한국어 형태소 분석 및 불용어 처리
# ============================================
print("=" * 60)
print("3. 형태소 분석기 설정...")
print("=" * 60)

# Okt 형태소 분석기 초기화
okt = Okt()
# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()

# 한국어 불용어 리스트
korean_stopwords = [
    '의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자',
    '에', '와', '한', '하다', '을', '를', '에서', '으로', '로', '에게', '의해', '까지',
    '수', '것', '등', '및', '그', '저', '것', '또', '또한', '더', '매우', '정말'
]


def tokenize_korean(text):
    """한국어 토큰화 (명사 추출)"""
    # 명사만 추출
    nouns = okt.nouns(text)
    # nouns = kkma.nouns(text)
    # nouns = komoran.nouns(text)
    # nouns = hannanum.nouns(text)

    # 불용어 제거 및 2글자 이상만 선택
    tokens = [noun for noun in nouns if noun not in korean_stopwords and len(noun) > 1]
    return ' '.join(tokens)

print("형태소 분석기 설정 완료!\n")


3. 형태소 분석기 설정...
형태소 분석기 설정 완료!



In [ ]:
tokenize_korean(cleaned_texts[11])

'국내 최초 칫솔 살균 개발 설립 에네스 소프트 소프트웨어 개발 기업 이자 울트라 웨이브 브랜드 살균 초점 퍼스 스케 제품 개발 제조 수출 기업 독립채산제 운영 특성 회사 아이 사업 울트라 웨이브 사업 총괄 원래 아이 사업 보이스 레코 음향 기기 대부분 방식 납품 스마트폰 사용 보편화 음향 기기 시장 규모 친환경 반도체 광원 국내 최초 칫솔 살균 개발 기존 램프 칫솔 살균 램프 수은 살균 과정 오존 발생 기키 때문 비릿 냄새 단점 파장 사용 울트라 웨이브 칫솔 살균 뮤탄스균 녹농균 대장균 황색포도상구균 살균 수은 함유 친환경 오존 발생 불쾌감 전기 전자제품 유해 물질 사용제한 인증 취득 미국 승인 미국 인증 유럽 인증 완료 화상 상담 해외 시장 확대 가능성 다비 에네스 소프트 칫솔 살균 면도기 살균 마스크 살균 살균 라인 반려동물 질환 염증 치료 동물 의료기기 반려동물 전용 펫헬 스케 라인 구성 코로나 변수 코로나 참가 예정 해외 전시회 줄줄이 취소 상황 발생 해외 시장조사 신규 바이어 영업 위해 코트라 도움 요청 영국 스페인 독일 미국 싱가포르 홍콩 말레이시아 태국 칠레 여러 국가 기업 화상 상담 진행 해외 시장 확대 중소기업 신제품 개발 기업 해외 전시회 참여 필수 코로나 해외 출장 상황 화상 상담 희망이 진행 함부르크 무역 화상 상담 무엇 성과 화상 상담 에네스 소프트 함부르크 무역 제품 유통 화상 상담 제안 화상 상담 진행 쿠쿠 전자 유럽 업체 아마존 독일 유럽 온라인 플랫폼 한국 롯데 하이마트 오프라인 유통 라인 보유 화상 상담 덕분 독일 샘플 물량 추출 세계 퍼스 스케 살균 시장 울트라 웨이브 제품 대해 관심 대감 표현 제품 기능 위주 질문 답변 양사 추구 유통 방향 대해 서도 협의 테스트 물량 수출 에네스 소프트 현재 아마존 독일 론칭 준비 중이 독일 전체 제품 유통 대해 서도 논의 에네스 소프트 여러 차례 화상 상담 계기 하반기 달러 규모 신규 수출 예상 코로나 오히려 에네스 소프트 기회 코로나 이전 바이어 퍼스 스케 살균 시간 설득 코로나 대유

#### ============================================
#### 4. BERTopic 모델 설정
#### ============================================

In [ ]:
# ============================================
# 4. BERTopic 모델 설정
# ============================================
print("=" * 60)
print("4. BERTopic 모델 설정 중...")
print("=" * 60)

# 4-1. 한국어 임베딩 모델 선택
# 옵션 1: 다국어 모델 (추천)
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 옵션 2: 한국어 특화 모델 (더 좋은 성능을 원한다면)
#embedding_model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

# 임베딩 생성 (progress bar 표시)
# embeddings = embedding_model.encode(
#     cleaned_texts,
#     show_progress_bar=True,
#     batch_size=32  # 배치 크기 (메모리에 따라 조정)
# )
# print(f"임베딩 모델: {embedding_model}")

# 4-2. Vectorizer 설정 (형태소 분석 적용)
vectorizer_model = CountVectorizer(
    tokenizer=lambda x: x.split(),  # 이미 토큰화된 텍스트 사용
    min_df=2,  # 최소 2번 이상 출현한 단어만 사용
    max_df=0.95,  # 95% 이상의 문서에 나타난 단어 제외
     ngram_range=(1, 2)  # 1-gram과 2-gram 모두 사용
)

# add clustering
#cluster_model = KMeans(n_clusters=10)
#cluster_model = AgglomerativeClustering(n_clusters=10)

umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.0, metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)


# 4-3. BERTopic 모델 초기화
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,

    umap_model=umap_model,  ## 추가
    hdbscan_model=hdbscan_model, ## 추가
    #hdbscan_model=cluster_model, ## 추가

    language='korean',
    calculate_probabilities=True,
    verbose=True,
    # min_topic_size=5,  # 최소 토픽 크기 (데이터 크기에 따라 조정)
    # nr_topics='auto'  # 토픽 수 자동 결정
    nr_topics=6  # 토픽 수 지정

)

print("BERTopic 모델 설정 완료!\n")

4. BERTopic 모델 설정 중...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BERTopic 모델 설정 완료!



#### ============================================
#### 5. 토픽 모델링 실행
#### ============================================

In [ ]:
# ============================================
# 5. 토픽 모델링 실행
# ============================================
print("=" * 60)
print("5. 토픽 모델링 실행 중... (시간이 걸릴 수 있습니다)")
print("=" * 60)

# 형태소 분석 적용
print("형태소 분석 진행 중...")
tokenized_texts = [tokenize_korean(text) for text in cleaned_texts]

embeddings = embedding_model.encode(
    tokenized_texts,
    show_progress_bar=True,
    batch_size=32  # 배치 크기 (메모리에 따라 조정)
)

# 토픽 모델 학습
topics, probs = topic_model.fit_transform(tokenized_texts, embeddings)
#topics, probs = topic_model.fit_transform(tokenized_texts)

# Time Series by Topic
topics_over_time = topic_model.topics_over_time(tokenized_texts, timestamps)


print(f"\n토픽 모델링 완료!")
print(f"발견된 토픽 수: {len(set(topics)) - 1}개 (노이즈 제외)")

5. 토픽 모델링 실행 중... (시간이 걸릴 수 있습니다)
형태소 분석 진행 중...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

2025-10-16 08:57:23,125 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-16 08:57:51,510 - BERTopic - Dimensionality - Completed ✓
2025-10-16 08:57:51,513 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-16 08:57:51,549 - BERTopic - Cluster - Completed ✓
2025-10-16 08:57:51,550 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-10-16 08:57:52,133 - BERTopic - Representation - Completed ✓
2025-10-16 08:57:52,136 - BERTopic - Topic reduction - Reducing number of topics
2025-10-16 08:57:52,155 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-16 08:57:52,513 - BERTopic - Representation - Completed ✓
2025-10-16 08:57:52,516 - BERTopic - Topic reduction - Reduced number of topics from 8 to 6
6it [00:00,  9.79it/s]


토픽 모델링 완료!
발견된 토픽 수: 5개 (노이즈 제외)


In [ ]:
tokenized_texts[11]

'국내 최초 칫솔 살균 개발 설립 에네스 소프트 소프트웨어 개발 기업 이자 울트라 웨이브 브랜드 살균 초점 퍼스 스케 제품 개발 제조 수출 기업 독립채산제 운영 특성 회사 아이 사업 울트라 웨이브 사업 총괄 원래 아이 사업 보이스 레코 음향 기기 대부분 방식 납품 스마트폰 사용 보편화 음향 기기 시장 규모 친환경 반도체 광원 국내 최초 칫솔 살균 개발 기존 램프 칫솔 살균 램프 수은 살균 과정 오존 발생 기키 때문 비릿 냄새 단점 파장 사용 울트라 웨이브 칫솔 살균 뮤탄스균 녹농균 대장균 황색포도상구균 살균 수은 함유 친환경 오존 발생 불쾌감 전기 전자제품 유해 물질 사용제한 인증 취득 미국 승인 미국 인증 유럽 인증 완료 화상 상담 해외 시장 확대 가능성 다비 에네스 소프트 칫솔 살균 면도기 살균 마스크 살균 살균 라인 반려동물 질환 염증 치료 동물 의료기기 반려동물 전용 펫헬 스케 라인 구성 코로나 변수 코로나 참가 예정 해외 전시회 줄줄이 취소 상황 발생 해외 시장조사 신규 바이어 영업 위해 코트라 도움 요청 영국 스페인 독일 미국 싱가포르 홍콩 말레이시아 태국 칠레 여러 국가 기업 화상 상담 진행 해외 시장 확대 중소기업 신제품 개발 기업 해외 전시회 참여 필수 코로나 해외 출장 상황 화상 상담 희망이 진행 함부르크 무역 화상 상담 무엇 성과 화상 상담 에네스 소프트 함부르크 무역 제품 유통 화상 상담 제안 화상 상담 진행 쿠쿠 전자 유럽 업체 아마존 독일 유럽 온라인 플랫폼 한국 롯데 하이마트 오프라인 유통 라인 보유 화상 상담 덕분 독일 샘플 물량 추출 세계 퍼스 스케 살균 시장 울트라 웨이브 제품 대해 관심 대감 표현 제품 기능 위주 질문 답변 양사 추구 유통 방향 대해 서도 협의 테스트 물량 수출 에네스 소프트 현재 아마존 독일 론칭 준비 중이 독일 전체 제품 유통 대해 서도 논의 에네스 소프트 여러 차례 화상 상담 계기 하반기 달러 규모 신규 수출 예상 코로나 오히려 에네스 소프트 기회 코로나 이전 바이어 퍼스 스케 살균 시간 설득 코로나 대유

In [ ]:
print(topic_model.get_topic(1))
topic_model.get_topic_info()

[('요트', 0.03303644996515704), ('선박', 0.029176345867615756), ('부품', 0.028589995887614895), ('신우', 0.02285346821500836), ('엘리베이터', 0.022766668979443377), ('크레인', 0.02169720095440129), ('금지', 0.02138227678606976), ('배관', 0.02021652957481509), ('양호', 0.019885295883270225), ('특수', 0.019540505589933116)]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,20,-1_커피_컴퍼니_수입_살균,"[커피, 컴퍼니, 수입, 살균, 치과, 종자, 바르샤바, 홍삼, 파쇄, 로움]",[커피 용품 시장 선두 기업 레드 기업 소개 우리나라 커피 시장 급속도 확대 부터 ...
1,0,98,0_엔지니어링_투자_식품_건설,"[엔지니어링, 투자, 식품, 건설, 설비, 에스, 교육, 장비, 납품, 부품]",[폴란드 유럽 진출 교두보 세코 닉스 기업 소개 세코 닉스 플라스틱 광학 기술 기반...
2,1,25,1_요트_선박_부품_신우,"[요트, 선박, 부품, 신우, 엘리베이터, 크레인, 금지, 배관, 양호, 특수]",[배관 부품 전문 제조업체 항해 영남 메탈 터닝 포인트 바다 육지 바다 천직 기사 ...
3,2,18,2_유아_국수_피부_재활,"[유아, 국수, 피부, 재활, 뷰티, 유아 용품, 엄마, 여성, 닥터, 미용]",[품질 국산 유아 용품 세계 베스트 레이드 기업 소개 베스트 레이드 미국 유럽 패션...
4,3,17,3_대추_장갑_코트라 지사_우리 회사,"[대추, 장갑, 코트라 지사, 우리 회사, 형제, 이집트, 사람, 비누, 정부, 파나마]",[세계 곡물 색채 별기 도전 대원 가치 가슴 용교 구실 얼매 번의 정성 가을 추수 ...
5,4,16,4_마스크_피부_바이오_한류,"[마스크, 피부, 바이오, 한류, 염색, 에이치, 뷰티, 태국 비즈니스, 유럽 시장...",[피부 발견 코코넛 발효 시트 투자 전문가 이진한 대표 화장품 회사 설립 피부 방문...


In [ ]:
#topic_model.get_representative_docs(4)

#### ============================================
#### 6. 결과 분석
#### ============================================

In [ ]:
# ============================================
# 6. 결과 분석
# ============================================
print("\n" + "=" * 60)
print("6. 토픽 분석 결과")
print("=" * 60)

# 토픽별 문서 수
topic_counts = pd.Series(topics).value_counts().sort_index()
print("\n[토픽별 문서 수]")
for topic_id, count in topic_counts.items():
    if topic_id == -1:
        print(f"토픽 {topic_id} (노이즈): {count}개")
    else:
        print(f"토픽 {topic_id}: {count}개")

# 각 토픽의 상위 키워드
print("\n[각 토픽의 주요 키워드 (상위 10개)]")
for topic_id in sorted(set(topics)):
    if topic_id != -1:  # 노이즈 제외
        keywords = topic_model.get_topic(topic_id)
        print(f"\n토픽 {topic_id}:")
        for word, score in keywords[:10]:
            print(f"  - {word}: {score:.4f}")

# 토픽 정보 DataFrame
topic_info = topic_model.get_topic_info()
print("\n[토픽 정보 요약]")
print(topic_info)


6. 토픽 분석 결과

[토픽별 문서 수]
토픽 -1 (노이즈): 20개
토픽 0: 98개
토픽 1: 25개
토픽 2: 18개
토픽 3: 17개
토픽 4: 16개

[각 토픽의 주요 키워드 (상위 10개)]

토픽 0:
  - 엔지니어링: 0.0165
  - 투자: 0.0156
  - 식품: 0.0148
  - 건설: 0.0146
  - 설비: 0.0144
  - 에스: 0.0139
  - 교육: 0.0134
  - 장비: 0.0130
  - 납품: 0.0130
  - 부품: 0.0123

토픽 1:
  - 요트: 0.0330
  - 선박: 0.0292
  - 부품: 0.0286
  - 신우: 0.0229
  - 엘리베이터: 0.0228
  - 크레인: 0.0217
  - 금지: 0.0214
  - 배관: 0.0202
  - 양호: 0.0199
  - 특수: 0.0195

토픽 2:
  - 유아: 0.0424
  - 국수: 0.0325
  - 피부: 0.0302
  - 재활: 0.0300
  - 뷰티: 0.0276
  - 유아 용품: 0.0262
  - 엄마: 0.0234
  - 여성: 0.0214
  - 닥터: 0.0208
  - 미용: 0.0198

토픽 3:
  - 대추: 0.0498
  - 장갑: 0.0470
  - 코트라 지사: 0.0340
  - 우리 회사: 0.0325
  - 형제: 0.0294
  - 이집트: 0.0232
  - 사람: 0.0230
  - 비누: 0.0215
  - 정부: 0.0215
  - 파나마: 0.0207

토픽 4:
  - 마스크: 0.0652
  - 피부: 0.0412
  - 바이오: 0.0352
  - 한류: 0.0329
  - 염색: 0.0254
  - 에이치: 0.0245
  - 뷰티: 0.0232
  - 태국 비즈니스: 0.0228
  - 유럽 시장: 0.0224
  - 코리안: 0.0208

[토픽 정보 요약]
   Topic  Count                  Name  \
0     -1     2

#### ============================================
#### 7. 시각화 1: 원본 임베딩 사용
#### ============================================
#### ============================================
#### 8. 시각화 2: UMAP 차원 축소 후 시각화
#### ============================================

In [ ]:
# ============================================
# 7. 시각화 1: 원본 임베딩 사용
# ============================================
print("\n" + "=" * 70)
print("7단계: 문서 시각화 (원본 고차원 임베딩)")
print("=" * 70)

try:
    # 원본 임베딩으로 시각화 (내부적으로 UMAP 적용)
    fig1 = topic_model.visualize_documents(
        cleaned_texts,
        embeddings=embeddings,
        hide_annotations=False,
        hide_document_hover=False,
        custom_labels=True
    )

    # HTML 파일로 저장
    fig1.write_html('bertopic_visualization_original.html')
    print("✓ 시각화 1 완료: 'bertopic_visualization_original.html' 저장")
    print("  → 원본 고차원 임베딩을 사용한 문서 분포")

except Exception as e:
    print(f"⚠ 시각화 1 생성 중 오류: {e}")


# ============================================
# 8. 시각화 2: UMAP 차원 축소 후 시각화
# ============================================
print("\n" + "=" * 70)
print("8단계: UMAP 차원 축소 후 시각화 (더 빠른 반복 분석)")
print("=" * 70)

try:
    # UMAP으로 차원 축소 (고차원 → 2차원)
    print("✓ UMAP 차원 축소 진행 중...")
    reduced_embeddings = UMAP(
        n_neighbors=10,      # 이웃 수
        n_components=2,      # 2차원으로 축소
        min_dist=0.0,        # 최소 거리
        metric='cosine',     # 코사인 유사도
        random_state=42      # 재현성
    ).fit_transform(embeddings)

    print(f"✓ 차원 축소 완료: {embeddings.shape} → {reduced_embeddings.shape}")

    # 축소된 임베딩으로 시각화 (빠른 속도)
    fig2 = topic_model.visualize_documents(
        tokenized_texts,
        reduced_embeddings=reduced_embeddings,
        hide_annotations=False,
        hide_document_hover=False,
        custom_labels=True
    )

    # HTML 파일로 저장
    fig2.write_html('bertopic_visualization_umap.html')
    print("✓ 시각화 2 완료: 'bertopic_visualization_umap.html' 저장")
    print("  → UMAP 차원 축소를 적용한 문서 분포 (빠른 렌더링)")

except Exception as e:
    print(f"⚠ 시각화 2 생성 중 오류: {e}")


# ============================================
# 9. 추가 시각화
# ============================================
print("\n" + "=" * 70)
print("9단계: 추가 시각화")
print("=" * 70)

try:
    # 토픽 간 거리 시각화
    fig3 = topic_model.visualize_topics()
    fig3.write_html('topic_distance_map.html')
    print("✓ 토픽 거리 맵: 'topic_distance_map.html'")

    # 토픽 바차트
    fig4 = topic_model.visualize_barchart(top_n_topics=15, n_words=10)
    fig4.write_html('topic_barchart.html')
    print("✓ 토픽 바차트: 'topic_barchart.html'")

    # 토픽 계층 구조
    hierarchical_topics = topic_model.hierarchical_topics(tokenized_texts)
    fig5 = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
    fig5.write_html('topic_hierarchy.html')
    print("✓ 토픽 계층 구조: 'topic_hierarchy.html'")

    # 토픽 히트맵
    fig6 = topic_model.visualize_heatmap()
    fig6.write_html('topic_heatmap.html')
    print("✓ 토픽 유사도 히트맵: 'topic_heatmap.html'")

except Exception as e:
    print(f"⚠ 일부 추가 시각화 생성 중 오류: {e}")


7단계: 문서 시각화 (원본 고차원 임베딩)
✓ 시각화 1 완료: 'bertopic_visualization_original.html' 저장
  → 원본 고차원 임베딩을 사용한 문서 분포

8단계: UMAP 차원 축소 후 시각화 (더 빠른 반복 분석)
✓ UMAP 차원 축소 진행 중...
✓ 차원 축소 완료: (194, 384) → (194, 2)
✓ 시각화 2 완료: 'bertopic_visualization_umap.html' 저장
  → UMAP 차원 축소를 적용한 문서 분포 (빠른 렌더링)

9단계: 추가 시각화
✓ 토픽 거리 맵: 'topic_distance_map.html'
✓ 토픽 바차트: 'topic_barchart.html'


100%|██████████| 4/4 [00:00<00:00, 193.42it/s]


✓ 토픽 계층 구조: 'topic_hierarchy.html'
✓ 토픽 유사도 히트맵: 'topic_heatmap.html'


In [ ]:

# ============================================
# 10. 결과 저장
# ============================================
print("\n" + "=" * 60)
print("10. 결과 저장 중...")
print("=" * 60)

# 원본 데이터에 토픽 정보 추가
df_with_topics = df[df['bdtCntnt'].notna()].copy()
df_with_topics['topic'] = topics
df_with_topics['topic_probability'] = [prob.max() for prob in probs]

# 결과 저장
# df_with_topics.to_csv('bertopic_results.csv', index=False, encoding='utf-8-sig')
# print("결과가 'bertopic_results.csv'에 저장되었습니다.")

# 토픽 정보 저장
topic_info.to_csv('topic_info.csv', index=False, encoding='utf-8-sig')
print("토픽 정보가 'topic_info.csv'에 저장되었습니다.\n")


10. 결과 저장 중...
토픽 정보가 'topic_info.csv'에 저장되었습니다.



In [ ]:
topic_model.visualize_distribution(topic_model.probabilities_[11])

- Time Series by Topic

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time)
#topic_model.visualize_topics_over_time(topics_over_time, topics=[0, 1, 2, 3, 4, 5, 6, 7])

- Hierarchical Topic Clustering

In [ ]:
topic_model.visualize_hierarchy()
#topic_model.visualize_heatmap()

- Topic Word

In [ ]:
topic_model.visualize_barchart(top_n_topics=10, n_words=10)

In [ ]:
# # ============================================
# # 11. 추가 분석 함수
# # ============================================

# def find_similar_documents(topic_model, text, top_n=5):
#     """특정 텍스트와 유사한 문서 찾기"""
#     tokenized = tokenize_korean(preprocess_korean_text(text))
#     similar_topics, similarity = topic_model.find_topics(tokenized, top_n=top_n)
#     return similar_topics, similarity

# def get_representative_docs(topic_model, topic_id, n_docs=3):
#     """특정 토픽의 대표 문서 가져오기""
#     return topic_model.get_representative_docs(topic_id)[:n_docs]

# # 사용 예시
# print("\n[추가 분석 예시]")
# print("특정 토픽의 대표 문서를 보려면:")
# print("docs = get_representative_docs(topic_model, topic_id=0, n_docs=3)")

In [ ]:
# docs = get_representative_docs(topic_model, topic_id=1, n_docs=3)
# docs[1]